In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from al_rosenbrock_experiment import run_experiment

In [3]:
# ['nngp', 'mcdue', 'random']
config = {
    'estimators': ['nngp', 'mcdue', 'random'],
    'random_seed': 43,
    'n_dim': 10,
    'data_size': 2000,
    'data_split': [0.2, 0.1, 0.1, 0.6],
    'update_size': 100,
    'al_iterations': 10,
    'verbose': True,
    'use_cache': True,
    'layers': [10, 128, 64, 32, 1],
    'patience': 5
}